In [2]:
from osgeo import gdal
import numpy as np
import os
import scipy.ndimage
import scipy.stats
import matplotlib.pyplot as plt
import time
import skimage.measure


In [3]:
def make_raster(in_ds, fn, data, data_type, nodata=None): 
    """Create a one-band GeoTIFF.
    in_ds     - datasource to copy projection and geotransform from
    fn        - path to the file to create
    data      - NumPy array containing data to write
    data_type - output data type
    nodata    - optional NoData value
    """
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(fn, in_ds.RasterXSize, in_ds.RasterYSize, 1, data_type)
    out_ds.SetProjection(in_ds.GetProjection()) 
    out_ds.SetGeoTransform(in_ds.GetGeoTransform())
    out_band = out_ds.GetRasterBand(1)
    if nodata is not None:
        out_band.SetNoDataValue(nodata)
        out_band.WriteArray(data)
        out_band.FlushCache()
        out_band.ComputeStatistics(False)
    return out_ds

In [4]:
# Write a function that calculates slope using a 3x3 window.
# This will be passed to the SciPy filter function below.
def slope(data, cell_width, cell_height):
    """Calculates slope using a 3x3 window.

    data        - 1D array containing the 9 pixel values, starting
                  in the upper left and going left to right and down
    cell_width  - pixel width in the same units as the data
    cell_height - pixel height in the same units as the data
    """
    rise = ((data[6] + (2 * data[7]) + data[8]) - (data[0] + (2 * data[1]) + data[2])) / (8 * cell_height)
    run =  ((data[2] + (2 * data[5]) + data[8]) - (data[0] + (2 * data[3]) + data[6])) / (8 * cell_width)
    dist = np.sqrt(np.square(rise) + np.square(run))
    return np.arctan(dist) * 180 / np.pi

In [5]:
# Write a function that calculates 3D area of DEM using a 3x3 window.
# This will be passed to the SciPy filter function below.
def area3D(data, cell_width, cell_height):
    """Calculates area per cell using a 3x3 window and a triangulated regular network.

    data        - 1D array containing the 9 pixel values, starting
                  in the upper left and going left to right and down
    cell_width  - pixel width in the same units as the data
    cell_height - pixel height in the same units as the data
    """
    
    xyz1 = np.array([0*cell_height,1*cell_width,data[1]])
    xyz2 = np.array([0*cell_height,2*cell_width,data[2]])
    xyz3 = np.array([1*cell_height,0*cell_width,data[3]])
    xyz4 = np.array([1*cell_height,1*cell_width,data[4]])
    xyz5 = np.array([1*cell_height,2*cell_width,data[5]])
    xyz6 = np.array([2*cell_height,0*cell_width,data[6]])
    xyz7 = np.array([2*cell_height,1*cell_width,data[7]])
    
    base12 = np.linalg.norm(xyz1 - xyz2)
    base34 = np.linalg.norm(xyz3 - xyz4)
    base45 = np.linalg.norm(xyz4 - xyz5)
    base67 = np.linalg.norm(xyz6 - xyz7)
    
    height14 = np.linalg.norm(xyz1 - xyz4)
    height25 = np.linalg.norm(xyz2 - xyz5)
    height36 = np.linalg.norm(xyz3 - xyz6)
    height47 = np.linalg.norm(xyz4 - xyz7)
    
    t124 = base12*height14/2
    t134 = base34*height14/2
    t245 = base45*height25/2
    t346 = base34*height36/2
    t457 = base45*height47/2
    t467 = base67*height47/2
    
    area = 0.25*(t124+t245+t346+t467)+0.5*(t134+t457)
    
    return area

In [6]:
#in_fn = '/Users/opizarro/data/FK20180119/bathy/hawaii_bty_5m_Au_au_0-400m.tif' # low res version - not 5m
in_fn = '/Users/opizarro/data/FK20180119/bathy/au_au_clipped_UTM_4N.tif'

In [7]:
# test section of input data
in_ds = gdal.Open(in_fn)

off_ulx = 3300
off_uly = 2000
columns = 1000
rows = 800
in_data = in_ds.GetRasterBand(1).ReadAsArray(off_ulx, off_uly, columns, rows)


#cell_width = 5
cell_width = in_ds.GetGeoTransform()[1]
#cell_height = 5 
cell_height = in_ds.GetGeoTransform()[5]
print("cell height ",cell_height)
print("input data shape ",in_data.shape)

('cell height ', -4.9997952662983325)
('input data shape ', (800, 1000))


In [8]:
# wrapper functon to allow for different window sizes


        
def fit_plane_local_a(data, cell_width, cell_height,Pinv):
    
    theta = np.dot(Pinv,data)
    a = theta[0]
    return a

def fit_plane_local_b(data, cell_width, cell_height,Pinv):
    
    theta = np.dot(Pinv,data)
    b = theta[1]
    return b

def fit_plane_window(in_data, win_size, cell_width, cell_height ):
    
    dim = win_size
    npoints = dim**2
    M = np.ones((npoints,3)) 
    k = 0
    for i in range(dim):
        for j in range(dim):
            M[k,:] = [j*cell_width, i*cell_height, 1]
            k+=1

    Pinv = np.linalg.pinv(M)        

    a = scipy.ndimage.filters.generic_filter(in_data, fit_plane_local_a, size=win_size, mode='nearest', extra_arguments=(
        cell_width, cell_height, Pinv))
    
    b = scipy.ndimage.filters.generic_filter(in_data, fit_plane_local_b, size=win_size, mode='nearest', extra_arguments=(
        cell_width, cell_height, Pinv))
    
    # plane equation is ax + by +c = z
    # or a*(j*cw) + b*(i*ch) + c = z
    slope = np.arctan(np.sqrt(np.square(a)+np.square(b)))*180/np.pi
    aspect = np.arctan2(b,a)*180/np.pi
    # area per horizontal unit area
    areaphua = np.sqrt(np.square(a)+np.square(b)+np.ones(a.shape))
    print(slope.shape)
    print(aspect.shape)
        
    return (slope, aspect, areaphua, a, b)


In [9]:
# calculate 3D area per cell
tic = time.time()
area3 = scipy.ndimage.filters.generic_filter(in_data, area3D, size=3, mode='nearest', extra_arguments=(
        cell_width, cell_height))
toc = time.time()
print("shape of area3 ",area3.shape)
print("ellapsed time ",toc-tic)
plt.figure(1)
plt.imshow(area3)
plt.draw()
plt.show(block=False)


('shape of area3 ', (800, 1000))
('ellapsed time ', 38.17685317993164)


In [10]:
scales = []
feat = []
for win_size in [3,5,7,9]:
    print("window size ", win_size)
    tic = time.time()
    slope, aspect, areaphua, a, b = fit_plane_window(in_data, win_size, cell_width, cell_height)
   
    area3Dwin = scipy.ndimage.filters.uniform_filter(area3,win_size)
    rugosity = np.divide(area3Dwin,areaphua*cell_width*cell_height)
    
    toc = time.time()
    
    scales.append((win_size,slope, aspect, rugosity, area3Dwin, areaphua, a, b))
    
    feat.append(slope)
    feat.append(aspect)
    feat.append(rugosity)
    #feat.append(np.stack((slope, aspect, rugosity), axis=-1))
    #print("feat shape " ,np.stack((slope, aspect, rugosity), axis=-1).shape)
    print("shape of slope ", slope.shape)
    print("ellapsed time ", toc-tic)
    plt.figure(win_size*5)
    plt.imshow(slope)
    plt.title('slope %s'%(win_size))
    
#    plt.figure(win_size*5+1)
#    plt.imshow(aspect, cmap="hsv")
#    plt.title('aspect %s'%(win_size))
    
#    plt.figure(win_size*5+2)
#    plt.imshow(rugosity)
#    plt.title('rugosity %s'%(win_size))
    
#    plt.figure(win_size*5+3)
#    plt.imshow(area3Dwin)
#    plt.title('area3Dwin %s'%(win_size))
    
#    plt.figure(win_size*5+4)
#    plt.imshow(areaphua)
#    plt.title('areaphua %s'%(win_size))
    
    
plt.show(block=False)

# add depth data
feat.append(in_data)
# stack list in 3 dimensional array
featstack = np.stack(feat,axis=-1)

('window size ', 3)
(800, 1000)
(800, 1000)
('shape of slope ', (800, 1000))
('ellapsed time ', 1.9843149185180664)
('window size ', 5)
(800, 1000)
(800, 1000)
('shape of slope ', (800, 1000))
('ellapsed time ', 2.1659579277038574)
('window size ', 7)
(800, 1000)
(800, 1000)
('shape of slope ', (800, 1000))
('ellapsed time ', 2.1493959426879883)
('window size ', 9)
(800, 1000)
(800, 1000)
('shape of slope ', (800, 1000))
('ellapsed time ', 2.2540881633758545)


In [11]:
#print("in_data shape ",in_data.shape)
#feat= []
#feat.append(in_data)


print("feature stack shape ",featstack.shape)

plt.figure(5)
nplots = featstack.shape[2]
f, faxarr = plt.subplots(4,1)
g, gaxarr = plt.subplots(4,1)
h, haxarr = plt.subplots(4,1)


for k in range(4):
    faxarr[k].hist(featstack[:,:,3*k].flatten(),bins=50)
    gaxarr[k].hist(featstack[:,:,3*k+1].flatten(),bins=50)
    haxarr[k].hist(featstack[:,:,3*k+2].flatten(),bins=np.arange(0.9,1.5,0.02))

plt.show(block=False)




('feature stack shape ', (800, 1000, 13))


In [12]:

import spectral

# test on just depth data
proc_data=in_data[:,:,np.newaxis]

print( "proc_data shape", proc_data.shape )
num_clusters = 15
classes, centres = spectral.kmeans(proc_data, num_clusters, 40)
print("classes shape ", classes.shape)
plt.figure(10)
plt.imshow(classes, cmap = "nipy_spectral")
plt.figure(11)
plt.imshow(in_data, cmap = "nipy_spectral" )
plt.show(block=False)

('proc_data shape', (800, 1000, 1))
Initializing clusters along diagonal of N-dimensional bounding box.
Iteration 1...  0.0%Iteration 1...799999 pixels reassigned.
Iteration 2...  0.0%Iteration 2...55420 pixels reassigned.
Iteration 3...  0.0%Iteration 3...43625 pixels reassigned.
Iteration 4...  0.0%Iteration 4...36471 pixels reassigned.
Iteration 5...  0.0%Iteration 5...29847 pixels reassigned.
Iteration 6...  0.0%Iteration 6...23750 pixels reassigned.
Iteration 7...  0.0%Iteration 7...21999 pixels reassigned.
Iteration 8...  0.0%Iteration 8...19411 pixels reassigned.
Iteration 9...  0.0%Iteration 9...16075 pixels reassigned.
Iteration 10...  0.0%Iteration 10...13725 pixels reassigned.
Iteration 11...  0.0%Iteration 11...11603 pixels reassigned.
Iteration 12...  0.0

In [13]:
# downsample to make problem more tractable


def mode_win(data,axis):
    return scipy.stats.mstats.mode(data,axis)[0]

from skimage.measure import block_reduce
tic = time.time()
#mode_data = scipy.ndimage.filters.generic_filter(classes, mode_win, size = 11)
mode_data = block_reduce(classes, block_size=(10,10), func = mode_win )
toc = time.time()
print("mode block time ",toc-tic)
print("mode_data shape", mode_data.shape)

('mode block time ', 18.314656972885132)
('mode_data shape', (80, 100, 10, 1))


In [14]:

plt.figure(13)
plt.imshow(mode_data[:,:,0,0], cmap = "nipy_spectral")
plt.show(block=False)

In [15]:
# region labeling with scikit
label_image, rnum = skimage.measure.label(mode_data[:,:,0,0],connectivity=2,return_num=True)
print("number of regions ", rnum)
plt.figure(113)
plt.imshow(label_image, cmap = "prism")
plt.show(block=False)

('number of regions ', 565)


In [16]:
 
tic = time.time()
properties = skimage.measure.regionprops(label_image,mode_data[:,:,0,0])
toc = time.time()
print("properties calc time {} for {} regions ".format(toc-tic,rnum))
print("size of properties {}".format(len(properties)))
print(properties[500].centroid)

properties calc time 0.00438499450684 for 565 regions 
size of properties 565
(60.0, 54.0)


In [17]:
# plot centroids and create a list with cluster_ids for each region

plt.figure(114)
plt.imshow(label_image, cmap = "prism")
label_list = []
clusterid_list = []

for ri in range(len(properties)):
    plt.scatter(x=properties[ri].centroid[1],y=properties[ri].centroid[0],c='k',s=10)
    label_list.append(properties[ri].label)
    clusterid_list.append(int(properties[ri].mean_intensity))
plt.show(block=False)
    

In [18]:
# generate distance matrix
Dmat = np.zeros((len(properties),len(properties)))
for i in range(len(properties)):
    for j in range(len(properties)):
        Dmat[i,j]=np.linalg.norm(np.array(properties[i].centroid)-np.array(properties[j].centroid))
        

In [19]:
from ortools.constraint_solver import pywrapcp
# You need to import routing_enums_pb2 after pywrapcp!
from ortools.constraint_solver import routing_enums_pb2

print(np.array(properties[0].centroid)-np.array(properties[1].centroid))
print(np.linalg.norm(np.array(properties[0].centroid)-np.array(properties[1].centroid)))

def Distance_slow(i, j):
    global properties
    return np.linalg.norm((np.array(properties[i].centroid)-np.array(properties[j].centroid)))

def Distance(i,j):
    global Dmat
    global depot
    if i == depot or j == depot:
        d = 0
    else:
        d = Dmat[i,j]
    return d

[ 1.5  -0.25]
1.52069063257


In [20]:
# assemble generalised TSP

tsp_size = rnum+1 # num of locations plus one for dummy depot node
print("tsp_size", tsp_size)
num_vehicles = 1
depot = tsp_size-1 # last node is a dummy node
routing = pywrapcp.RoutingModel(tsp_size, num_vehicles, depot)

search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
    # Setting first solution heuristic (cheapest addition).
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Setting the cost function.
    # Put a callback to the distance accessor here. The callback takes two
    # arguments (the from and to node inidices) and returns the distance between
    # these nodes
routing.SetArcCostEvaluatorOfAllVehicles(Distance)

('tsp_size', 566)


In [21]:
# disjunctions based on cluster ID 
# np.where expects an np array
cluster_modes = np.array(clusterid_list)
print("max cluster_modes ", max(cluster_modes))
print("length cluster_modes ",len(cluster_modes))
#print(np.where(cluster_modes==5))
#nodes = np.where(cluster_modes ==5)
#print(nodes)

#print(list(nodes[0]))
#print(nodes[0])
print(cluster_modes)
for i in range(num_clusters):
    #np.where returns a tuple with an np array as the first element and AddDisjunction expects a list
    nodes = list(np.where(cluster_modes == i)[0])
    # remove depot node (typicall 0) (as default depo)
    if depot in nodes:
        nodes.remove(depot)
        print("removed depot node ",depot)
    print(np.where(cluster_modes == i))
    if (len(nodes) > 0):
        routing.AddDisjunction(nodes)
        print("cluster ", i)
        print("length ",len(nodes))
        print("max id", max(nodes))
    else:
        print("no nodes for clusterid ",i)
    
  

('max cluster_modes ', 14)
('length cluster_modes ', 565)
[ 6  8  9 10  8  6  5  6  8  9 12 10  9 11 12  9  8  6  5  4  5  6  5  7  8
  7  9 10  9 11  5  9  7  7 11  7  7  7  8 13 11  8  9  8  7  7 11  7  7  7
  5  6  4  6  8  6  7  6  9  9  8  7  8 10  3  3 11 10 11  5  5  7  9  9  5
  3  6  4  7 11  5  6  7  8  6  5  7  7  8  9  8 11  5  6  9 10  8  7 11 12
  9  6  1  2  8  7  6  8  9 11 10  5  7  9  6  3  7  8  6 10  7  8  5  2  1
  2  5  6  9  9 10 10  7  5  5  2  4  5  6  7  4  5 11  2  1  8  6  6  8  6
  3  8  2  7  9 10 10  7  5  4  2  4  7  5  8  2  9  3  8  4  8 11  6  4  8
 10  7  7  1  2  7  9 10  8  4  5  4  7  8  4  4  5  6  8 10  5  6  8  7  6
 12 10  9  6  5  6  7  9  9 11  9  8  6  5  7 11  9  7  6  7  8  9 12  9 10
  4  9  9  7 10  8 12 11  7 10  8 12  8 11 12 11  9  7 11 13 10 10  8 12 12
 10 12 13 13 14 13 11 10 12 10  9 10 10 10 12 13 13 14 11 12 10 11 12 12 13
  9 13 12 13 12 13 13 12 12 10 11 12 13 13 12 13 13 12 13 11 12 10 11 12 12
 13 11 13 13 11 10 11 12 11 12

In [42]:

# Solve, returns a solution if any.
tic = time.time()
assignment = routing.SolveWithParameters(search_parameters)
toc = time.time()

print("solving tsp time ", toc-tic)    


('solving tsp time ', 0.3560519218444824)


In [41]:
      


    
#assignment = routing.Solve()

In [52]:
# modified to not show dummy depot node
#
solnodes = []
print("solving tsp time ", toc-tic)
if assignment:
    print "Total distance of all routes: " + str(assignment.ObjectiveValue()) + "\n"
    for vehicle_nbr in range(num_vehicles):
        index = routing.Start(vehicle_nbr)
        index_next = assignment.Value(routing.NextVar(index))
        route = ''
        route_dist = 0
        route_demand = 0

        while not routing.IsEnd(index_next):
            node_index = routing.IndexToNode(index)
            node_index_next = routing.IndexToNode(index_next)
            if node_index != depot:
                route += str(node_index) + " -> "
            # Add the distance to the next node.
            route_dist += Distance(node_index, node_index_next)
            # Add demand.
            #route_demand += demands[node_index_next]
            index = index_next
            index_next = assignment.Value(routing.NextVar(index))
            solnodes.append(node_index)
            
        node_index = routing.IndexToNode(index)
        node_index_next = routing.IndexToNode(index_next)
        if node_index_next != depot:
            route += str(node_index) + " -> " + str(node_index_next)
        else:
            route += str(node_index)
        solnodes.append(node_index)
        route_dist += Distance(node_index, node_index_next)
        print "Route for vehicle " + str(vehicle_nbr) + ":\n\n" + route + "\n"
        print "Distance of route " + str(vehicle_nbr) + ": " + str(route_dist)
        #print "Demand met by vehicle " + str(vehicle_nbr) + ": " + str(route_demand) + "\n"
else:
    print 'No solution found.'


    

('solving tsp time ', 0.3560519218444824)
Total distance of all routes: 117

Route for vehicle 0:

541 -> 474 -> 504 -> 489 -> 471 -> 402 -> 433 -> 377 -> 267 -> 213 -> 225 -> 165 -> 115 -> 102

Distance of route 0: 125.401647503


In [62]:
xsol = []
ysol = []
for i in solnodes[1:-1]:
    print(i)
    xsol.append(properties[i].centroid[1])
    ysol.append(properties[i].centroid[0])
print(xsol)
plt.figure(13)
plt.plot(xsol,ysol,'k',lw=2)
plt.show(block=False)
    

541
474
504
489
471
402
433
377
267
213
225
165
115
[82.0, 77.134788189987162, 69.400000000000006, 67.5, 63.0, 62.07692307692308, 64.0, 67.0, 78.245161290322585, 68.5, 66.5, 55.0, 54.399999999999999]


In [43]:
if assignment:
    # Solution cost.
    print(assignment.ObjectiveValue())
    # Inspect solution.
    # Only one route here; otherwise iterate from 0 to routing.vehicles() - 1
    route_number = 0
    node = routing.Start(route_number)
    route = ''
    while not routing.IsEnd(node):
        route += str(node) + " -> "
        node = assignment.Value(routing.NextVar(node))
    route += '0'
    print(route)
else:
    print('No solution found.')

117
565 -> 541 -> 474 -> 504 -> 489 -> 471 -> 402 -> 433 -> 377 -> 267 -> 213 -> 225 -> 165 -> 115 -> 102 -> 0
